In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import csv

%matplotlib inline

In [2]:
def normal(B):
    B = [x - min(B) for x in B]
    
    return ([x/(max(B)-min(B)) for x in B])

In [3]:
def f(t):
    if t>(6/29)**3:
        return t**(1/3)
    else:
        return (t*(1/3)*(29/6)**2)+(4/29)

In [4]:
def Lab(X,Y,Z,Xn,Yn,Zn):
    t_x = X/Xn
    t_y = Y/Yn
    t_z = Z/Zn
    
    L = 116*f(t_y) - 16
    a = 500*(f(t_x)-f(t_y))
    b = 200*(f(t_y)-f(t_z))
    
    return L, a, b

In [14]:
def CH(a,b):
    C = np.sqrt(a**2+b**2)
    h = np.arctan(b/a)
    return C,h

In [5]:
wl=[]
LED=[]
inc=[]
with open('lightsources.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names {",".join(row)}')
            line_count += 1
        else:
            wl.append(float(row[0]))
            LED.append(float(row[3]))
            inc.append(float(row[5]))
            line_count += 1
    print(f'Processed {line_count} lines.')
    
LED = normal(LED)
inc = normal(inc)

Column names ï»¿fluoorescent2: Wavelength (nm),fluoorescent2: Intensity (rel),LED1: Intensity (rel),LED2: Intensity (rel),incandescent1: Intensity (rel),incandescent2: Intensity (rel),fluorescent1: Intensity (rel),,,,
Processed 501 lines.


In [6]:
wlmcc=[]
m=np.zeros([81,24])
with open('mcc.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names {",".join(row)}')
            line_count += 1
        else:
            wlmcc.append(float(row[0]))
            for i in range(24):
                m[line_count-1,i]=float(row[i])
            line_count += 1
    print(f'Processed {line_count} lines.')

Column names ï»¿wavelength,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,,
Processed 82 lines.


In [7]:
wlCIE=[]
x_=[]
y_=[]
z_=[]
with open('CIE.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names {",".join(row)}')
            line_count += 1
        else:
            wlCIE.append(float(row[0]))
            x_.append(float(row[1]))
            y_.append(float(row[2]))
            z_.append(float(row[3]))
            line_count += 1
    print(f'Processed {line_count} lines.')

Column names ï»¿(nm),xÂ¯Î»,yÂ¯Î»,zÂ¯Î»
Processed 58 lines.


In [8]:
nx_ = interp1d(wlCIE, x_, kind='linear')
nx_ = nx_(wl)

ny_ = interp1d(wlCIE, y_, kind='linear')
ny_ = ny_(wl)

nz_ = interp1d(wlCIE, z_, kind='linear')
nz_ = nz_(wl)

#

XnL = sum(np.multiply(LED,nx_))
YnL = sum(np.multiply(LED,ny_))
ZnL = sum(np.multiply(LED,nz_))

XnI = sum(np.multiply(inc,nx_))
YnI = sum(np.multiply(inc,ny_))
ZnI = sum(np.multiply(inc,nz_))

In [9]:
nmcc = np.zeros([500,24])
for i in range(24):
    nm = interp1d(wlmcc, m[:,i], kind='linear')
    nmcc[:,i] = nm(wl)


patchII = np.zeros([500,24])    
XI = np.zeros(24)    
YI = np.zeros(24)    
ZI = np.zeros(24)    

patchIL = np.zeros([500,24])    
XL = np.zeros(24)    
YL = np.zeros(24)    
ZL = np.zeros(24)    


for i in range(24):
    patchIL[:,i] = np.multiply(nmcc[:,i],LED)
    patchII[:,i] = np.multiply(nmcc[:,i],inc)
    
    XL[i] = sum(np.multiply(patchIL[:,i],nx_))
    YL[i] = sum(np.multiply(patchIL[:,i],ny_))
    ZL[i] = sum(np.multiply(patchIL[:,i],nz_))
    
    XI[i] = sum(np.multiply(patchII[:,i],nx_))
    YI[i] = sum(np.multiply(patchII[:,i],ny_))
    ZI[i] = sum(np.multiply(patchII[:,i],nz_))

In [16]:
IL = np.zeros(24)  
Ia = np.zeros(24)  
Ib = np.zeros(24)  

LL = np.zeros(24)  
La = np.zeros(24)  
Lb = np.zeros(24)  

CI = np.zeros(24)  
hI = np.zeros(24)  

CL = np.zeros(24)  
hL = np.zeros(24)  

for i in range(24):
    IL[i],Ia[i],Ib[i] = Lab(XI[i],YI[i],ZI[i],XnI,YnI,ZnI)
    LL[i],La[i],Lb[i] = Lab(XL[i],YL[i],ZL[i],XnL,YnL,ZnL)

    CI[i], hI[i] = CH(Ia[i],Ib[i])
    CL[i], hL[i] = CH(La[i],Lb[i])

In [23]:
dECIE = np.zeros(24)
for i in range(24):
    dECIE[i] = np.sqrt(((LL[i]-IL[i])**2)+((Lb[i]-Ib[i])**2)+((La[i]-Ia[i])**2))
    
for i in range(24):
    if dECIE[i]<2.3:
        print(i)

5
19
20
21
22
23


In [24]:
dECH = np.zeros(24)
for i in range(24):
    dECH[i] = np.sqrt(((LL[i]-IL[i])**2)+((CL[i]-CI[i])**2)+((hL[i]-hI[i])**2))
    
for i in range(24):
    if dECH[i]<2.3:
        print(i)

5
20
21
22
23
